In [51]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/basilwong/awstest1
   a7de9ed..1204a0f  master     -> origin/master
Updating a7de9ed..1204a0f
Fast-forward
 ...dio_file_1586672026.7589955.mp3_accompaniment.wav | Bin 10584058 -> 0 bytes
 .../audio_file_1586672026.7589955.mp3_vocals.wav     | Bin 10584058 -> 0 bytes
 ...dio_file_1586672072.6487582.mp3_accompaniment.wav | Bin 10584058 -> 0 bytes
 .../audio_file_1586672072.6487582.mp3_vocals.wav     | Bin 10584058 -> 0 bytes
 ...dio_file_1586672118.4082892.mp3_accompaniment.wav | Bin 10584058 -> 0 bytes
 .../audio_file_1586672118.4082892.mp3_vocals.wav     | Bin 10584058 -> 0 bytes
 ...dio_file_1586672164.2195466.mp3_accompaniment.wav | Bin 10584058 -> 0 bytes
 .../audio_file_1586672164.2195466.mp3_vocals.wav     | Bin 10584058 -> 0 bytes
 .

### Add Dependencies

In [64]:
import sagemaker as sage
from sagemaker import get_execution_role

import zipfile
import os

from sagemaker import ModelPackage


# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('src')

!pip install pydub

import audio_util

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [23]:
# Execution role
role = get_execution_role()
# S3 prefixes
common_prefix = "source_separation"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
# Sagemaker Session
sagemaker_session = sage.Session()
# Arn for Source Separator Model Package
modelpackage_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/source-separation-v11570291536-75ed8128ecee95e142ec4404d884ecad'



### Creating the Model

In [29]:
from sagemaker import ModelPackage

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='application/x-recordio-protobuf')

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

### Running the Batch Job

In [27]:
batch_input_folder = "source-separation-input"


transform_input = sagemaker_session.upload_data(batch_input_folder, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-075178354542/source_separation/batch-inference-input-data


In [30]:
import json 
import uuid

bucket = sagemaker_session.default_bucket()

transformer = model.transformer(1, 'ml.m4.xlarge', strategy='SingleRecord', output_path='s3://'+bucket+'/'+common_prefix+'/batch-transform-output')
transformer.transform(transform_input, content_type='application/x-recordio-protobuf')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

....................Starting the inference server with 4 workers.
[2020-04-12 06:13:02 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2020-04-12 06:13:02 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2020-04-12 06:13:02 +0000] [10] [INFO] Using worker: gevent
[2020-04-12 06:13:02 +0000] [14] [INFO] Booting worker with pid: 14
[2020-04-12 06:13:02 +0000] [15] [INFO] Booting worker with pid: 15
[2020-04-12 06:13:02 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-12 06:13:02 +0000] [17] [INFO] Booting worker with pid: 17
Testing...
Testing...
2020-04-12 06:13:46.428785: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
169.254.255.130 - - [12/Apr/2020:06:13:46 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:13:46 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586672026

169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:06:20:05 +0000] "POST /invocations HTTP/1.1" 200 4379039 "-" "Go-http-client/1.1"

Batch Transform output saved to s3://sagemaker-us-east-2-075178354542/source_separation/batch-transform-output


### Processing the Batch Output

In [52]:
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'source-separation-output/batch-transform-output/output-{}.zip'.format(i))

input1.mp3.out
input2.mp3.out
input3.mp3.out
input4.mp3.out
input5.mp3.out
input6.mp3.out
input7.mp3.out
input8.mp3.out
input9.mp3.out


In [66]:

audio_util.clear_folder('source-separation-output/batch-transform-output')
for file in os.listdir('source-separation-output/batch-transform-output'):
    print(file)
    with zipfile.ZipFile('source-separation-output/batch-transform-output/'+file, 'r') as zip_ref:
        zip_ref.extractall('source-separation-output/extracted/'+file.split('.')[0]+'/')